In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_expanded_SMILES.csv')

In [5]:
df = df[~df['passivating_molecule_SMILES'].isna()]

In [7]:
import numpy as np
import re

def parse_perovskite_formula(formula):
    # Define allowed species (order matters for multi-letter elements)
    allowed_species = ["FA", "MA", "CS", "Rb", "Pb", "Sn", "I", "Br", "Cl"]

    # if is the nan we return component dictionary with all zeros
    if formula is np.nan:
        formula = ""    
    
    # Dictionary to store parsed results (initialize with 0.0 for all species)
    parsed_result = {species: 0.0 for species in allowed_species}

    # Step 1: Handle groups in parentheses with coefficients (e.g., (FAPbI3)0.95)
    pattern_group = r"\(([^)]+)\)\s*([0-9\.]+)"

    
    
    groups = re.findall(pattern_group, formula)

    if groups:
        for group, coef in groups:
            coef = float(coef)  # Convert coefficient to float
            elements = re.findall(r"(FA|MA|CS|Rb|Pb|Sn|I|Br|Cl)\s*([\d\.]*)", group)
            for element, count in elements:
                count = float(count) if count else 1.0
                parsed_result[element] += count * coef  # Distribute coefficient

    # Step 2: Handle formulas without parentheses (e.g., FA1-xMAxPbI3)
    remaining_formula = re.sub(r"\([^)]*\)\s*[0-9\.]+", "", formula)  # Remove processed groups
    elements = re.findall(r"(FA|MA|CS|Rb|Pb|Sn|I|Br|Cl)\s*([\d\.]*)", remaining_formula)

    for element, count in elements:
        count = float(count) if count and 'x' not in count else 1.0  # Ignore '-x' or 'x'
        parsed_result[element] += count

    # Round to 2 decimal places for all values
    parsed_result = {k: round(v, 2) for k, v in parsed_result.items()}

    return parsed_result

# Test cases
formulas = [
    "(FAPbI3)0.95(MAPbBr3)0.05",
    "FA1-xMAxPbI3",
    "FA0.9CS0.1Rb0.05PbI2.9Br0.1",
    "(CS0.8Rb0.2FAPbI3)0.9(MAPbBr3)0.1",
    "(C4H9NH3)2PbI 4"  # Test case with space
]

for formula in formulas:
    print(f"Formula: {formula}")
    print("Parsed:", parse_perovskite_formula(formula))
    print()

Formula: (FAPbI3)0.95(MAPbBr3)0.05
Parsed: {'FA': 0.95, 'MA': 0.05, 'CS': 0.0, 'Rb': 0.0, 'Pb': 1.0, 'Sn': 0.0, 'I': 2.85, 'Br': 0.15, 'Cl': 0.0}

Formula: FA1-xMAxPbI3
Parsed: {'FA': 1.0, 'MA': 1.0, 'CS': 0.0, 'Rb': 0.0, 'Pb': 1.0, 'Sn': 0.0, 'I': 3.0, 'Br': 0.0, 'Cl': 0.0}

Formula: FA0.9CS0.1Rb0.05PbI2.9Br0.1
Parsed: {'FA': 0.9, 'MA': 0.0, 'CS': 0.1, 'Rb': 0.05, 'Pb': 1.0, 'Sn': 0.0, 'I': 2.9, 'Br': 0.1, 'Cl': 0.0}

Formula: (CS0.8Rb0.2FAPbI3)0.9(MAPbBr3)0.1
Parsed: {'FA': 0.9, 'MA': 0.1, 'CS': 0.72, 'Rb': 0.18, 'Pb': 1.0, 'Sn': 0.0, 'I': 2.7, 'Br': 0.3, 'Cl': 0.0}

Formula: (C4H9NH3)2PbI 4
Parsed: {'FA': 0.0, 'MA': 0.0, 'CS': 0.0, 'Rb': 0.0, 'Pb': 1.0, 'Sn': 0.0, 'I': 4.0, 'Br': 0.0, 'Cl': 0.0}



In [11]:
temp = df['perovskite_composition'].apply(parse_perovskite_formula).apply(pd.Series)
df = df.join(temp)

In [21]:
df.shape

(34, 29)

In [25]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

def compute_molecular_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return [
                Descriptors.MolWt(mol),  # Molecular weight
                Descriptors.ExactMolWt(mol),  # Exact molecular weight (isotope-specific)
                Descriptors.MolLogP(mol),  # LogP (lipophilicity)
                Descriptors.TPSA(mol),  # Topological Polar Surface Area
                Descriptors.NumValenceElectrons(mol),  # Total valence electrons
                rdMolDescriptors.CalcNumRotatableBonds(mol),  # Rotatable bonds
                rdMolDescriptors.CalcNumHBA(mol),  # Hydrogen bond acceptors
                rdMolDescriptors.CalcNumHBD(mol),  # Hydrogen bond donors
                rdMolDescriptors.CalcFractionCSP3(mol),  # Fraction of sp3 carbons
                rdMolDescriptors.CalcNumAromaticRings(mol),  # Number of aromatic rings
                rdMolDescriptors.CalcNumSaturatedRings(mol),  # Number of saturated rings
                rdMolDescriptors.CalcNumHeteroatoms(mol),  # Number of heteroatoms
                rdMolDescriptors.CalcNumHeavyAtoms(mol),  # Number of heavy atoms
                rdMolDescriptors.CalcNumSpiroAtoms(mol),  # Number of spiro atoms
                rdMolDescriptors.CalcNumBridgeheadAtoms(mol),  # Number of bridgehead atoms
                Descriptors.FpDensityMorgan1(mol),  # Morgan fingerprint density (radius=1)
                Descriptors.FpDensityMorgan2(mol),  # Morgan fingerprint density (radius=2)
                Descriptors.FpDensityMorgan3(mol),  # Morgan fingerprint density (radius=3)
                Descriptors.qed(mol),  # Quantitative Estimate of Drug-likeness
                rdMolDescriptors.CalcNumLipinskiHBA(mol),  # Lipinski Hydrogen Bond Acceptors
                rdMolDescriptors.CalcNumLipinskiHBD(mol),  # Lipinski Hydrogen Bond Donors
                rdMolDescriptors.CalcNumRings(mol),  # Total number of rings
                rdMolDescriptors.CalcNumAmideBonds(mol),  # Number of amide bonds
                Descriptors.BalabanJ(mol),  # Balaban’s connectivity index
                Descriptors.BertzCT(mol),  # Bertz complexity
                Descriptors.Chi0(mol),  # Chi connectivity index (order 0)
                Descriptors.Chi1(mol),  # Chi connectivity index (order 1)
                Descriptors.Chi2n(mol),  # Chi connectivity index (order 2, non-H)
                Descriptors.Kappa1(mol),  # Kappa Shape Index (order 1)
                Descriptors.Kappa2(mol),  # Kappa Shape Index (order 2)
            ]
        else:
            return [np.nan] * 30  # Return NaN for missing values
    except:
        return [np.nan] * 30  # Return NaN for exceptions

# Apply function to dataset
mol_features = df['passivating_molecule_SMILES'].apply(compute_molecular_features)

# Convert list to DataFrame
mol_features_df = pd.DataFrame(mol_features.tolist(), 
                               columns=[
                                   'MolWt', 'ExactMolWt', 'LogP', 'TPSA', 'NumValenceElectrons',
                                   'NumRotBonds', 'NumHBA', 'NumHBD', 'FractionCSP3', 'AromaticRings',
                                   'SaturatedRings', 'Heteroatoms', 'HeavyAtoms', 'SpiroAtoms', 
                                   'BridgeheadAtoms', 'FpDensityMorgan1', 'FpDensityMorgan2', 
                                   'FpDensityMorgan3', 'QED', 'LipinskiHBA', 
                                   'LipinskiHBD', 'NumRings', 'NumAmideBonds', 'BalabanJ', 
                                   'BertzCT', 'Chi0', 'Chi1', 'Chi2n', 'Kappa1', 'Kappa2'
                               ],
                               index=df.index)

# Merge with original dataset
df = pd.concat([df, mol_features_df], axis=1)

Index(['first_num', 'perovskite_composition', 'electron_transport_layer',
       'hole_transport_layer', 'structure_pin_nip', 'test', 'stability_type',
       'passivating_molecule', 'humidity', 'temperature', 'time',
       'control_pce', 'treated_pce', 'control_voc', 'treated_voc',
       'efficiency_control', 'efficiency_tret', 'efficiency_cont',
       'passivating_molecule_cleaned', 'passivating_molecule_SMILES', 'FA',
       'MA', 'CS', 'Rb', 'Pb', 'Sn', 'I', 'Br', 'Cl', 'MolWt', 'ExactMolWt',
       'LogP', 'TPSA', 'NumValenceElectrons', 'NumRotBonds', 'NumHBA',
       'NumHBD', 'FractionCSP3', 'AromaticRings', 'SaturatedRings',
       'Heteroatoms', 'HeavyAtoms', 'SpiroAtoms', 'BridgeheadAtoms',
       'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'QED',
       'LipinskiHBA', 'LipinskiHBD', 'NumRings', 'NumAmideBonds', 'BalabanJ',
       'BertzCT', 'Chi0', 'Chi1', 'Chi2n', 'Kappa1', 'Kappa2'],
      dtype='object')

In [35]:
df.to_csv('df.csv')

In [ ]:
df = pd.DataFrame(

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
file_path = "df.csv"
df = pd.read_csv(file_path)

# Drop unnecessary columns
df = df.drop(columns=["Unnamed: 0", "first_num", 'test', 'perovskite_composition', 'passivating_molecule'])

# Define target variable
target = "treated_pce"

# Drop rows where the target variable is NaN
df_clean = df.dropna(subset=[target])

# Separate features and target variable
X_clean = df_clean.drop(columns=[target])
y_clean = df_clean[target]

# Identify categorical and numerical columns
categorical_cols = X_clean.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X_clean.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Handle NaNs in categorical columns by treating them as a separate category
cat_imputer = SimpleImputer(strategy="constant", fill_value="Missing")
cat_encoder = OneHotEncoder(handle_unknown="ignore")

# Handle NaNs in numerical columns by imputing with the median
num_imputer = SimpleImputer(strategy="median")
num_scaler = StandardScaler()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", num_imputer), ("scaler", num_scaler)]), numerical_cols),
        ("cat", Pipeline([("imputer", cat_imputer), ("encoder", cat_encoder)]), categorical_cols),
    ]
)

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create full pipeline
pipeline = Pipeline([("preprocessor", preprocessor), ("model", model)])

# Split the cleaned data
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Train model
pipeline.fit(X_train, y_train)

# Predictions
y_pred = pipeline.predict(X_test)

# Evaluate model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display evaluation metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Mean Absolute Error (MAE): 1.75
Root Mean Squared Error (RMSE): 2.03
R-squared (R²): 0.56


/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['efficiency_control']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['efficiency_control']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
